In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.13.0'

In [3]:
df = pd.read_csv('Churn_Modelling.csv')

In [8]:
X = df.iloc[:, 3:-1].values
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [10]:
y = df.iloc[: , -1].values
print(y)

[1 0 1 ... 1 1 0]


In [13]:
#encoding categorical data GENDER
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [15]:
#using onehotencoder on geography
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [16]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [19]:
X_train, X_test, y_train, y_test

(array([[0.0, 0.0, 1.0, ..., 1, 0, 163830.64],
        [0.0, 1.0, 0.0, ..., 1, 1, 57098.0],
        [1.0, 0.0, 0.0, ..., 1, 0, 185630.76],
        ...,
        [1.0, 0.0, 0.0, ..., 1, 0, 181429.87],
        [0.0, 0.0, 1.0, ..., 1, 1, 148750.16],
        [0.0, 1.0, 0.0, ..., 1, 0, 118855.26]], dtype=object),
 array([[0.0, 1.0, 0.0, ..., 1, 1, 192852.67],
        [1.0, 0.0, 0.0, ..., 1, 0, 128702.1],
        [0.0, 0.0, 1.0, ..., 1, 1, 75732.25],
        ...,
        [0.0, 0.0, 1.0, ..., 1, 0, 141533.19],
        [0.0, 1.0, 0.0, ..., 1, 1, 11276.48],
        [0.0, 1.0, 0.0, ..., 1, 0, 192950.6]], dtype=object),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 1, 0, ..., 0, 0, 0]))

In [21]:
#feature scaling
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

In [23]:
#initialize an ann
ann = tf.keras.models.Sequential()

In [24]:
#creating hidden layers
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [25]:
#2 hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [26]:
#output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [27]:
#compiling the ann
ann.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
#training the ann
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5656 - accuracy: 0.7588
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4479 - accuracy: 0.8092
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4237 - accuracy: 0.8156
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4127 - accuracy: 0.8184
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4034 - accuracy: 0.8253
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3932 - accuracy: 0.8342
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3838 - accuracy: 0.8394
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3769 - accuracy: 0.8444
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3707 - accuracy: 0.8472
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3653 - accura

In [30]:
testing on new data
'''
Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000
'''
print(ann.predict(ss.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 115ms/step
[[False]]


In [32]:
#testing on test data
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 3ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[1519   76]
 [ 199  206]]


0.8625